# FaaS over Fog and Cloud

## Prelude

### Imports

In [36]:
from os import makedirs

import plotly.io as pio
import polars as pl

from util import *

### Configuration

In [37]:
pl.Config.set_fmt_str_lengths(60)
pio.templates.default = "plotly_white"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create output dirs

In [38]:
makedirs("out/ddnn2", exist_ok=True)

### Load the data

In [39]:
variants = ["", "3", "5"]
raw_fission = [pl.read_ndjson(f"../ddnn2/benchmark/fission{e}-newdeploy-min1.json").lazy() for e in variants]
raw_openfaas = [pl.read_ndjson(f"../ddnn2/benchmark/openfaas{e}.json").lazy() for e in variants]
raw_kafkaml = [pl.read_ndjson(f"../ddnn2/benchmark/kafkaml{e}-fix.json").lazy() for e in variants]

fission = [normalize_data(x).collect() for x in raw_fission]
openfaas = [normalize_data(x).collect() for x in raw_openfaas]
kafkaml = [
    normalize_data_kafkaml(raw_kafkaml[0], 1682071612.1225128, clients=1).collect(),
    normalize_data_kafkaml(raw_kafkaml[1], 1682072010.837343, clients=3).collect(),
    normalize_data_kafkaml(raw_kafkaml[2], 1682072307.7277372, clients=5).collect(),
]

## Statistical analysis

In [40]:
divided_fission = list(map(divide_by_topics, fission))
divided_openfaas = list(map(divide_by_topics, openfaas))
divided_kafkaml = list(map(divide_by_topics, kafkaml))

stats_fission = list(map(get_stats, fission))
stats_openfaas = list(map(get_stats, openfaas))
stats_kafkaml = list(map(get_stats, kafkaml))

elapsed_fission = list(map(elapsed, fission))
elapsed_openfaas = list(map(elapsed, openfaas))
elapsed_kafkaml = list(map(elapsed, kafkaml))

### Single client

In [41]:
stats = pl.concat((stats_openfaas[0], stats_fission[0]))
stats.write_csv("out/ddnn2/stats_single.csv")
stats

topic,max,min,avg,median,std,samples
str,f64,f64,f64,f64,f64,u32
"""openfaas.inference.fog-output""",0.596291,0.215235,0.231943,0.225212,0.029204,200
"""openfaas.inference.cloud-output""",1.45512,0.402956,0.464654,0.458078,0.0785,200
"""fission.inference.cloud-output""",0.804382,0.407078,0.489413,0.476644,0.061393,200
"""fission.inference.fog-output""",0.562904,0.219454,0.260205,0.238377,0.060021,200


In [42]:
pl.DataFrame({
    "platform": ["openfaas", "fission", "kafkaml"],
    "elapsed": [elapsed_openfaas[0], elapsed_fission[0], elapsed_kafkaml[0]]
}).sort("elapsed")

platform,elapsed
str,f64
"""openfaas""",213.125
"""kafkaml""",214.464
"""fission""",223.215


In [43]:
fig = plot_latency_box({
    "openfaas fog": divided_openfaas[0]["fog"],
    "fission fog": divided_fission[0]["fog"],
    "kafkaml fog": divided_kafkaml[0]["fog"],
    "openfaas cloud": divided_openfaas[0]["cloud"],
    "fission cloud": divided_fission[0]["cloud"],
    "kafkaml cloud": divided_kafkaml[0]["cloud"],
})#.update_layout(showlegend=False)
fig.write_image("out/ddnn2/box_single.pdf")
fig

In [44]:
fig = plot_request_latency(
    {f"openfaas {k}": v for k, v in divided_openfaas[0].items()} |
    {f"fission {k}": v for k, v in divided_fission[0].items()} |
    {f"kafkaml {k}": v for k, v in divided_kafkaml[0].items()}
)#.update_layout(showlegend=False)
fig.write_image("out/ddnn2/latency_single.pdf")
fig

In [45]:
kafkaml_last = kafkaml[0].select(pl.col("timestamp") - pl.col("timestamp").min()).max()
fission_last = fission[0].select(pl.col("timestamp") - pl.col("timestamp").min()).max()
pl.DataFrame({
    "platform": ["kafkaml", "fission"],
    "timestamp": [kafkaml_last.item(), fission_last.item()],
})

platform,timestamp
str,f64
"""kafkaml""",214.464
"""fission""",223.215


### Three clients

In [46]:
stats = pl.concat((stats_openfaas[1], stats_fission[1]))
stats.write_csv("out/ddnn2/stats_three.csv")
stats

topic,max,min,avg,median,std,samples
str,f64,f64,f64,f64,f64,u32
"""openfaas.inference.fog-output""",22.069006,0.198611,4.387443,0.531778,6.033279,600
"""openfaas.inference.cloud-output""",25.200136,0.373268,6.109701,1.73824,7.218708,600
"""fission.inference.cloud-output""",25.827598,0.387704,6.035612,1.666875,7.204385,600
"""fission.inference.fog-output""",21.739489,0.190238,4.278183,0.539235,5.87955,600


In [47]:
pl.DataFrame({
    "platform": ["openfaas", "fission", "kafkaml"],
    "elapsed": [elapsed_openfaas[1], elapsed_fission[1], elapsed_kafkaml[1]]
}).sort("elapsed")

platform,elapsed
str,f64
"""kafkaml""",213.637
"""fission""",238.287
"""openfaas""",239.289


In [48]:
fig = plot_latency_box({
    "openfaas fog": divided_openfaas[1]["fog"],
    "fission fog": divided_fission[1]["fog"],
    "kafkaml fog": divided_kafkaml[1]["fog"],
    "openfaas cloud": divided_openfaas[1]["cloud"],
    "fission cloud": divided_fission[1]["cloud"],
    "kafkaml cloud": divided_kafkaml[1]["cloud"],
})#.update_layout(showlegend=False)
fig.write_image("out/ddnn2/box_three.pdf")
fig

In [49]:
fig = plot_request_latency(
    {f"openfaas {k}": v for k, v in divided_openfaas[1].items()} |
    {f"fission {k}": v for k, v in divided_fission[1].items()} |
    {f"kafkaml {k}": v for k, v in divided_kafkaml[1].items()}
)#.update_layout(showlegend=False)
fig.write_image("out/ddnn2/latency_three.pdf")
fig

### Five clients

In [50]:
stats = pl.concat((stats_openfaas[2], stats_fission[2]))
stats.write_csv("out/ddnn2/stats_five.csv")
stats

topic,max,min,avg,median,std,samples
str,f64,f64,f64,f64,f64,u32
"""openfaas.inference.fog-output""",98.288278,0.197524,29.323869,19.120906,29.952049,1000
"""openfaas.inference.cloud-output""",104.29578,0.388886,34.514531,24.872499,32.296763,1000
"""fission.inference.cloud-output""",92.860898,0.422054,30.602904,21.694511,28.608173,1000
"""fission.inference.fog-output""",86.524256,0.229448,25.52542,16.05994,26.194069,1000


In [51]:
pl.DataFrame({
    "platform": ["openfaas", "fission", "kafkaml"],
    "elapsed": [elapsed_openfaas[2], elapsed_fission[2], elapsed_kafkaml[2]]
}).sort("elapsed")

platform,elapsed
str,f64
"""fission""",315.884
"""openfaas""",318.392
"""kafkaml""",494.501


In [52]:
fig = plot_latency_box({
    "openfaas fog": divided_openfaas[2]["fog"],
    "fission fog": divided_fission[2]["fog"],
    "kafkaml fog": divided_kafkaml[2]["fog"],
    "openfaas cloud": divided_openfaas[2]["cloud"],
    "fission cloud": divided_fission[2]["cloud"],
    "kafkaml cloud": divided_kafkaml[2]["cloud"],
})
fig.write_image("out/ddnn2/box_five.pdf")
fig

In [53]:
fig = plot_request_latency(
    {f"openfaas {k}": v for k, v in divided_openfaas[2].items()} |
    {f"fission {k}": v for k, v in divided_fission[2].items()} |
    {f"kafkaml {k}": v for k, v in divided_kafkaml[2].items()}
)
fig.write_image("out/ddnn2/latency_five.pdf")
fig